# **CREACION DE UNA BASE DE DATOS ANALITICA**

### **EVIDENCIA DE APRENDISAJE 1**

**INTEGRANTES:**

**OSCAR LUIS MARQUEZ ARRIETA**

**NATALIA JADITH CASTRO OSPINO**

**PREICA2502B020061**

## **PROBLEMATICA**

La empresa de fluido electrico EnergyCol especializada en el servicio de energia electrica a gran parte de las regiones de colombia, tanto residenciales como comerciales. Esta esta presentando problemas con el procesamineto de los datos de todos los clientes debido al alto tamaño de informacion que se tienen de todos los clientes.

Debido al a esta problematica, se implementara una base de datos bien estructurada y organiza, que permita implementar toda la informacion que se requiera de los usuarios. Esto le brindara a la empresa una mejor organizacion y las relaciones entre el tamaño del edificio, la ocupación, la ubicación geográfica y los costos de energía.

## **Dataset**

**Fuente:** Kaggle: Residential and Commercial Energy Cost Dataset

**Enlace:** https://www.kaggle.com/datasets/andreylss/residential-and-commercial-energy-cost-dataset

# **VARIABLES RELEVANTES**

**id_cliente:** Identificador único para cada cliente

**tipo_cliente:** Tipo de propiedad (residencial o comercial)

**costo_energía_brl:** Costo mensual de energía en moneda local

**tipo_cliente:** Tipo de propiedad (residencial o comercial)

**regiones:** Región geográfica (Norte, Noreste, Medio Oeste, Sudeste, Sur)

**Este Dataset lo escogimos porque se complementa muy bien con los datos que tenemos de todos los usuarios de nuestar empresa. Estos nos facilita hacer una muy buena migracion a esta base datos muy organizada y funcional para todo lo que se necesita respecto a los datos de nuestros usuarios.**










# **Modelo Entidad-Relación (ERD)**

![Diagrama Entidad-Relación](./ERD_ENERGYCOL.png)

Este diagrama representa un modelo de base de datos relacional para gestionar el consumo de energía de la empres EnergyCol.

**REGION**


Contiene información sobre las regiones geográficas.


Campos:


- id_region: Identificador único de la región (PK).

- nombre_region: Nombre de la región.

- tarifa_promedio_brl: Tarifa promedio de energía en reales brasileños (BRL).



**CLIENTE**


Almacena los datos de los clientes.


Campos:


- id_cliente: Identificador único del cliente (PK).

- nombre_cliente: Nombre del cliente.

- tipo_cliente: Tipo de cliente (residencial, comercial, industrial, etc.).

- direccion: Dirección del cliente.

- id_region: Llave foránea que conecta con la tabla REGION (FK).




**CONSUMO_ENERGIA**


Registra el consumo mensual de energía de cada cliente.


Campos:


- id_consumo: Identificador único del registro de consumo (PK).

- id_cliente: Llave foránea que conecta con CLIENTE (FK).

- mes y anio: Período de consumo.

- consumo_kwh: Energía consumida en kilovatios-hora.

- costo_energia_brl: Costo correspondiente al consumo en reales.



**Relaciones:**


Una región puede tener muchos clientes.

Un cliente puede tener muchos registros de consumo de energía.



# **Creacion BD + Carga de datos**

In [0]:
!pip install kagglehub[pandas-datasets]>=0.3.8

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


## **Librerias implementadas**

In [0]:

import kagglehub
import pandas as pd
import os
import zipfile

## **Descargamos, extraemos y leemos el dataset de Kaggle**

In [0]:

def download_dataset_zip(url = ""):
        print("Descargando dataset desde Kaggle...")
        dataset_path = kagglehub.dataset_download(url)
        print("Ruta al dataset:", dataset_path)
        return dataset_path
    
def extract_zip_files(dataset_path):
        zip_files = [f for f in os.listdir(dataset_path) if f.endswith('.zip')]
        if zip_files:
            zip_file = os.path.join(dataset_path, zip_files[0])
            extract_dir = os.path.join(dataset_path, "extracted")
            os.makedirs(extract_dir, exist_ok=True)
            print(f"Extrayendo {zip_file} en {extract_dir}...")
            with zipfile.ZipFile(zip_file, "r") as z:
                z.extractall(extract_dir)
            return extract_dir
        else:
            # Si no se encuentra un ZIP, se verifica si existen archivos CSV en la ruta
            csv_files = [f for f in os.listdir(dataset_path) if f.endswith('.csv')]
            if csv_files:
                print("No se encontró archivo ZIP pero se detectaron archivos CSV; se asume que el dataset ya se encuentra extraído.")
                return dataset_path
            else:
                raise FileNotFoundError("No se encontró ningún archivo .zip ni archivos .csv en la ruta del dataset")

def create_csv(csv_dir):
        #os.makedirs('src/static/csv', exist_ok=True)
        csv_files = [f for f in os.listdir(csv_dir) if f.endswith('.csv')]
        if not csv_files:
            raise FileNotFoundError("No se encontraron archivos CSV en el directorio extraído")

        for file in csv_files:
            file_path = os.path.join(csv_dir, file)
            print(f"Leyendo {file_path}...")
            try:
                df = pd.read_csv(file_path, encoding="latin1")
            except Exception as e:
                print(f"Error al leer {file}: {e}")
                continue
            print(f"Creando/actualizando ")
        print("cvs creado correctamente en ")
        return df

## **Descargamos Dataset**

In [0]:
df = pd.DataFrame()
dataset = download_dataset_zip("andreylss/residential-and-commercial-energy-cost-dataset") 
csv_dir =extract_zip_files(dataset)
df = create_csv(csv_dir)


Descargando dataset desde Kaggle...


100%|██████████| 38.3k/38.3k [00:00<00:00, 6.48MB/s]

Extracting files...
Ruta al dataset: /home/spark-a9f2599d-a7de-48b1-90a8-ca/.cache/kagglehub/datasets/andreylss/residential-and-commercial-energy-cost-dataset/versions/1
No se encontró archivo ZIP pero se detectaron archivos CSV; se asume que el dataset ya se encuentra extraído.
Leyendo /home/spark-a9f2599d-a7de-48b1-90a8-ca/.cache/kagglehub/datasets/andreylss/residential-and-commercial-energy-cost-dataset/versions/1/energy_consumption.csv...
Creando/actualizando 
cvs creado correctamente en 


## **Verificamos la carga de los datos cargados de Kaggle**

In [0]:
df.head(3)

,customer_id,customer_type,regions,building_size_m2,occupants,energy_cost_brl
0,CUSTOMER_0001,residential,Northeast,24,2,64.51
1,CUSTOMER_0002,commercial,Midwest,24,1,55.26
2,CUSTOMER_0003,commercial,Southeast,24,1,74.54


## **Convertir Dataframe de pandas a Spark**

In [0]:

spark_df = spark.createDataFrame(df)

## **Creación la Tabla e Inserto de Datos**

In [0]:
spark_df.write.mode("overwrite").saveAsTable("tbl_clientes_al_dia")

## **Ver tabla creada**

In [0]:
%sql
SELECT
  *
FROM
  tbl_clientes_al_dia
LIMIT 5;

customer_id,customer_type,regions,building_size_m2,occupants,energy_cost_brl
CUSTOMER_0001,residential,Northeast,24,2,64.51
CUSTOMER_0002,commercial,Midwest,24,1,55.26
CUSTOMER_0003,commercial,Southeast,24,1,74.54
CUSTOMER_0004,residential,Northeast,45,4,147.06
CUSTOMER_0005,residential,Southeast,45,4,143.06


## **Cosultas SQL**

## **1. Conteo de registros**

In [0]:
%sql
SELECT
  COUNT(*)
FROM
  tbl_clientes_al_dia

COUNT(*)
5000


El conteo nos arroja que la tabla tbl_**clientes_al_dia** cuenta con 5000 registros en total realizados.

## **2. Nombre y tipos de columnas**

In [0]:
%sql
DESCRIBE TABLE tbl_clientes_al_dia

col_name,data_type,comment
customer_id,string,null
customer_type,string,null
regions,string,null
building_size_m2,bigint,null
occupants,bigint,null
energy_cost_brl,double,null


Esta cosulta nos devuelve la estructura de la tabala **tbl_clientes_al_dia**, nos muestra el nombre de la columna y el tipo de dato de cada uno de los datos.

## **3. Cosulta con filtro**

In [0]:
%sql
SELECT
  *
FROM
  tbl_clientes_al_dia
WHERE
  regions = 'Northeast'
LIMIT (10);

customer_id,customer_type,regions,building_size_m2,occupants,energy_cost_brl
CUSTOMER_0001,residential,Northeast,24,2,64.51
CUSTOMER_0004,residential,Northeast,45,4,147.06
CUSTOMER_0009,residential,Northeast,45,2,70.03
CUSTOMER_0011,residential,Northeast,52,3,65.47
CUSTOMER_0015,residential,Northeast,45,1,95.75
CUSTOMER_0016,residential,Northeast,17,1,90.83
CUSTOMER_0018,residential,Northeast,24,1,84.0
CUSTOMER_0027,residential,Northeast,77,4,112.98
CUSTOMER_0031,residential,Northeast,17,1,55.69
CUSTOMER_0035,commercial,Northeast,45,2,73.36


Aqui hacemos una cosulta a la tabla **tbl_clientes_al_dia** en la que hacemos un filtro en donde le mandamos que nos muestre a los clientes que estan en la region Nor oeste. Esta nos arroja muy claramente los clientes que se encuentran en esa region, nos muestra las filas y columnas de esta tabla.

In [0]:
%sql
SELECT
  COUNT(*)
FROM
  tbl_clientes_al_dia
WHERE
  regions = 'Northeast'

COUNT(*)
1266


Aqui quise realizar un conteo de cuantos clientes hay en la region Noroeste, este nos muestra que en total hay 1266 clientes en esta region.